In [ ]:
import time
import banpei
import pandas as pd
from model.anomaly import Anomaly
from .base_model_setting import BaseModelSetting
# if TYPE_CHECKING:
from model.time_series import TimeSeries

@dataclass
class SingularSpectrumTransformation(BaseModelSetting):
    '''
    annotation: the name of the colored region on plot
    color: color of the region on plot
    '''
    annotation: str
    color: str
    
    def anomalies(self, ts: TimeSeries) -> list['Anomaly']:
        '''
        Concrete implementation of BaseModelSetting.anomalies.
        Returns: list of `Anomaly` obj.
        TODO: terminate if model runs too long
        '''  

    def add_df_column(self, ts: TimeSeries):
        '''
        Add extra columns to the ts obj's DataFrame for plotting
        '''
        pass

    def window_size(self, ts: TimeSeries) -> int:
        return ts.period * self.num_periods

    def banpei(self, ts: TimeSeries) -> pd.Series:
        """
        Parameters:
        w    : int
               Window size
                   Window size for banpei is the # series between peak & trough
                   should be 1/2 of the window size produced in Auto-Period finder
        m    : int
               Number of basis vectors
        k    : int
               Number of columns for the trajectory and test matrices
        L    : int
               Lag time
        """
        anomaly_series = ts.anomaly_series
        # cal window size - banpei uses window as peak to trough
        window_size = int(self.window_size(ts)/2)
        # implementing Singular Spectrum Transformation (SST) model with window size parameter:
        model = banpei.SST(w=(window_size))
        # SST processing can be accelerated using the Lanczos method which part of Krylov subspace:
        results = model.detect(anomaly_series, is_lanczos=True)
        results.name = 'SST'
        return results
        
    def confidence_banpei(
            self,
            ts: TimeSeries,
            diff_series: pd.results) -> tuple[int, float]:
        '''
        The weight for the final ensemble: aka the degree of confidence:
        degree of confidence = (highest peak of residual) / (2nd higest peak of residual)
        Highest peak only since each dataset contains 1 anomoly.
        diff_series: the 2nd order diff series
        returns: tuple(index, confidence)
        '''
        conf = results['SST'].nlargest(2)
        largest = conf.T[conf.index[0]]
        conf = conf.T[conf.index[0]] / conf.T[conf.index[1]]
        return largest, conf